In [ ]:
import torch.nn as nn
import torch
import math
from torch.nn import functional as F

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader

import math
import copy

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
!pip install wandb
import wandb
wandb.login()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.7 MB/s eta 0:00:00


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
!pip install pymfe
from pymfe.mfe import MFE

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.0/156.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 21.9 MB/s eta 0:00:00


In [ ]:
torch.manual_seed(0)
np.random.seed(0)
import random
random.seed(0)

In [ ]:
# regressions
!wget -O random_regressions.csv https://www.dropbox.com/scl/fi/vka3aonbo9qlsh0mhtlvw/random_regressions.csv?rlkey=6ey79ntvfkb056kplnbmc5d6z&st=6fepsd26&dl=0

# fitted nets
!gdown 1w3qFkPECtwFnlaP-OPgM2NEn8mQENj0W
!yes | unzip "fitted 2.zip"
# !wget -O dataset.zip https://www.dropbox.com/scl/fo/17c1dgkhl1aqoxewqgzki/AAyM7JM9QxznSQbPlof4JOE?rlkey=rvwdpkn28uqxtku2cifi112d6&st=2sd8h56b&dl=0
# !yes | unzip "dataset.zip"

--2024-09-12 06:49:42--  https://www.dropbox.com/scl/fi/vka3aonbo9qlsh0mhtlvw/random_regressions.csv?rlkey=6ey79ntvfkb056kplnbmc5d6z
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6a1585dab403ccfdcda7393f30.dl.dropboxusercontent.com/cd/0/inline/CaZXzKtejMQbeqv_3gSmyzSasGT9AnZ3wnSas_N50lkm3xAtjTHQVbm80Q4ixoSnHzQ7oi1V6okInS1scAIvUyGVgmAl5pcjP2J_O84CkuqAfmauoSl0-GRVbuxNgfBqyid7PVtTw4xdb4W36igF3TfO/file# [following]
--2024-09-12 06:49:43--  https://uc6a1585dab403ccfdcda7393f30.dl.dropboxusercontent.com/cd/0/inline/CaZXzKtejMQbeqv_3gSmyzSasGT9AnZ3wnSas_N50lkm3xAtjTHQVbm80Q4ixoSnHzQ7oi1V6okInS1scAIvUyGVgmAl5pcjP2J_O84CkuqAfmauoSl0-GRVbuxNgfBqyid7PVtTw4xdb4W36igF3TfO/file
Resolving uc6a1585dab403ccfdcda7393f30.dl.dropboxusercontent.com (uc6a1585dab403ccfdcda7393f30.dl.dropboxusercontent.com)... 162.125.5.

In [ ]:
from random import randint, choice, random

act = {
    "ReLU": {
        "layer": nn.ReLU,
        "args": {},
    },
    "Tanh": {
        "layer": nn.Tanh,
        "args": {},
    },
    "Sigmoid": {
        "layer": nn.Sigmoid,
        "args": {},
    },
}

class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()

        self.layers = nn.Sequential()
        self.current_size = input_size

        self.count_layers = randint(1, 3)

        for i in range(self.count_layers):
            self.layers.append(self.make_linear())
            # if random() < .2: self.layers.append(self.make_normalization_layer())
            self.layers.append(self.make_activation_layer())

        self.layers.append(self.make_linear(output_size))

    def forward(self, x):
        return self.layers(x)

    def make_linear(self, output_size=None) -> nn.Linear:
        input_size = self.current_size

        if output_size is None:
            output_size = randint(4, 8)

        self.current_size = output_size

        return nn.Linear(in_features=input_size,
                         out_features=output_size,
                         bias=choice([True, False]))

    def make_activation_layer(self) -> nn.Module:
        layer_info = choice(list(act.values()))

        args = layer_info["args"]
        eval_args = {}
        for key, value in args.items():
            eval_args[key] = value() if callable(value) else value

        return layer_info["layer"](**eval_args)

    def make_normalization_layer(self) -> nn.Module:
        return choice([
            nn.BatchNorm1d(self.current_size),
            nn.LayerNorm(self.current_size),
        ])

In [ ]:
act_map = {nn.ReLU: 2,
           nn.Tanh: 3,
           nn.Sigmoid: 1}

def matrix_converter(net):

    matrix = torch.zeros(64, 64) + torch.full((64, 64), 0)
    count_matrix = torch.zeros(64, 64)
    dims = [0, 0]
    last_layer = 0
    layer_count = 1

    if net.__class__ != nn.Sequential:
        net = net.layers
    for i in range(len(net)):
        layer = net[i]
        if layer.__class__ in act_map.keys():
            for _ in range(last_layer):
                matrix[dims[0], dims[1]] = act_map[layer.__class__]
                count_matrix[dims[0], dims[1]] = layer_count
                dims[0] += 1
                dims[1] += 1
            layer_count += 1
        else:
            matrix[dims[0]:dims[0]+layer.out_features, dims[1]:dims[1]+layer.in_features] = layer.weight
            count_matrix[dims[0]:dims[0]+layer.out_features, dims[1]:dims[1]+layer.in_features] = layer_count * torch.eye(layer.out_features, layer.in_features)
            dims[0] += layer.out_features
            dims[1] += layer.in_features
            last_layer = layer.out_features
            layer_count += 1
    return matrix, count_matrix

In [ ]:
act_map = {nn.ReLU: 2,
           nn.Tanh: 3,
           nn.Sigmoid: 1}

def converter(net: nn.Module) -> torch.Tensor:
    matrix = torch.zeros((64, 64))
    x_p, y_p = 0, 0
    for layer in net.layers:
        if isinstance(layer, nn.Linear):
            matrix[x_p:x_p+layer.weight.shape[0], y_p:y_p+layer.weight.shape[1]] = layer.weight
            x_p += layer.weight.shape[0]
            y_p += layer.weight.shape[1]
        else:
            matrix[x_p, y_p] = act_map[layer.__class__]
            x_p += 1
            y_p += 1
    return matrix

def reverse_converter(weights: torch.Tensor, net: nn.Module) -> nn.Module: #
    x_p, y_p = 0, 0
    layers = []
    net_copy = copy.deepcopy(net)
    for layer in net_copy.layers:
        if isinstance(layer, nn.Linear):
            layer.weight = nn.Parameter(weights[x_p:x_p+layer.weight.shape[0], y_p:y_p+layer.weight.shape[1]])
            x_p += layer.weight.shape[0]
            y_p += layer.weight.shape[1]
        else:
            x_p += 1
            y_p += 1
        layers += [layer]
    return RNet(layers)

In [ ]:
class RNet(nn.Module):
    def __init__(self, in_layers):
        super().__init__()
        self.layers = nn.Sequential()
        for i in in_layers:
            self.layers.append(i)
    def forward(self, x):
        self.emb = self.layers[:-1](x)
        return self.layers[-1](self.emb)

In [ ]:
N_REGS = 4
N_NETS = 156
REGS_BATCHSIZE = 1
NETS_BATCHSIZE = 12

In [ ]:
regs = pd.read_csv('/content/random_regressions.csv')
uniq_regs = regs["reg_id"].unique()
reg_data = [regs[regs["reg_id"] == i] for i in uniq_regs]

In [ ]:
class RegDataset(Dataset):
    def __init__(self, df):
        super().__init__()
        self.features = torch.tensor(df.drop(columns=['reg_id', 'y']).values, dtype=torch.float32)
        self.targets = torch.tensor(df['y'].values, dtype=torch.float32)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [ ]:
reg_datasets = []
reg_dataloaders = []
for reg in reg_data[:N_REGS]:
    train_data, valid_data = train_test_split(reg, test_size=0.2)
    train_dataset = RegDataset(train_data)
    valid_dataset = RegDataset(valid_data)
    reg_datasets += [[train_dataset, valid_dataset]]
    train_loader = DataLoader(train_dataset, batch_size=REGS_BATCHSIZE, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=REGS_BATCHSIZE, shuffle=False)
    reg_dataloaders += [[train_loader, valid_loader]]

In [ ]:
class NetDataset(Dataset):
    def __init__(self, start, end):
        self.nets_num = []
        self.converted_nets = []
        self.regs = []
        for i in range(N_REGS):
            for j in range(start, end):
                net = torch.load(f"/content/fitted 2/reg{i}/model{j}.pt", map_location=torch.device('cpu'))
                converted_net = converter(net)
                self.converted_nets += [converted_net.unsqueeze(0)]

                self.regs += [i]
                self.nets_num += [j]

    def __len__(self):
        return len(self.nets_num)

    def __getitem__(self, idx):
        return self.converted_nets[idx], self.nets_num[idx], self.regs[idx]

def collate_fn(data):
    nets = torch.empty(NETS_BATCHSIZE, 1, 64, 64)
    nets_num = torch.empty(NETS_BATCHSIZE, 1)
    reg = torch.empty(NETS_BATCHSIZE, 1)

    for i, elem in enumerate(data):
        nets[i] = torch.tensor(elem[0])
        nets_num[i] = torch.tensor(elem[1])
        reg[i] = torch.tensor(elem[2])
    return nets, nets_num, reg

In [ ]:
# df = pd.read_csv('/content/models.csv')
# train, val = train_test_split(df, train_size=0.8)

# train_dataset = NetDataset(0, 79)
# valid_dataset = NetDataset(80, 99)

train_dataset = NetDataset(0, 120)
valid_dataset = NetDataset(120, 156)

train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=NETS_BATCHSIZE, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, collate_fn=collate_fn, batch_size=NETS_BATCHSIZE, shuffle=False)

<ipython-input-14-c7d25f251431>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net = torch.load(f"/content/fitted 2/reg{i}/model{j}.pt", map_location=torch.device('cpu'))


In [ ]:
train_dataset[0][0][0][:7, :10]

tensor([[-0.3604, -0.1105, -0.7219, -0.3692, -0.1126,  0.1502, -0.6595,  0.1718,
         -0.0256, -0.3288],
        [-0.3940, -0.0869,  0.2298,  0.1490, -0.5923, -0.2522,  0.5530, -0.3136,
          0.1687,  0.0712],
        [ 0.4279,  0.1517,  0.7527,  0.3645,  0.1853, -0.1404,  0.7168, -0.1719,
         -0.0055,  0.3787],
        [-0.7234, -0.1973, -0.5761, -0.4872, -0.2679, -0.1609, -0.4157, -0.2212,
         -0.1417, -0.3668],
        [-0.6290, -0.3690, -0.2903, -0.1558, -0.3791, -0.1682, -0.7425, -0.0055,
         -0.0339, -0.4633],
        [ 0.7029,  0.1985,  0.6042,  0.5124,  0.2223,  0.1443,  0.4214,  0.2209,
          0.1419,  0.3493],
        [ 0.5729,  0.3452,  0.2265,  0.1318,  0.3741,  0.1963,  0.6697,  0.0611,
          0.0286,  0.4340]], grad_fn=<SliceBackward0>)

In [ ]:
train_dataset[0][0][0][7:17, 10:20]

tensor([[ 1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00, -3.9568e+00,  4.4296e+00,  6.0010e+00, -4.0556e+00,
         -3.9639e+00,  5.9573e+00,  6.1152e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00, -4.0400e+00,  4.4464e+00,  6.1591e+00, -3.9307e+00,
         -4.0949e+00,  6.2050e+00,  6.1655e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  7.1633e+00,  5.0245e+00, -4.0747e+00,  7.1329e+00,
          7.2179e+00, -4.0448e+00, -4.1928e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00, -7.0786e-02, -2.0360e-02, -7.6265e-02, -1.2522e-01,
          1.0618e-01, -7.3601e-02, -1.4004e-01,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00, -4.0225e+00,  4.4956e+00,  6.0377e+00, -3.8789e+00,
         -3.9639e+00,  5.8627e+00,  5.9487e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  7.2157e+00,  5.0064e+00, -3.9432e+00,  7.2653e+00,
          7.2453e+00, -4.2499e+0

In [ ]:
for i in train_dataloader:
    print(i[0].shape)
    break

torch.Size([12, 1, 64, 64])


<ipython-input-14-c7d25f251431>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  nets[i] = torch.tensor(elem[0])


In [ ]:
len(train_dataset), len(valid_dataset)

(480, 144)

In [ ]:
class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

    def forward(self, x):
        return self.conv_block(x)

class Up(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)

    def forward(self, x1):
        x1 = self.up(x1)
        return x1

In [ ]:
class CNNEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.down0 = nn.Conv2d(1, 4, 3, padding=1)
        self.down1 = (Down(4, 8))
        self.down2 = (Down(8, 16))

        self.fc_sigma = nn.Linear(4096, 4096)
        self.fc_mu = nn.Linear(4096, 4096)
        self.N = torch.distributions.Normal(0, 1)
        self.N.loc = self.N.loc.to(device)
        self.N.scale = self.N.scale.to(device)

        self.kl = 0
    def forward(self, x):
        x1 = self.down0(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x_flatten = x3.reshape(x.shape[0], -1)

        # mu = self.fc_mu(x_flatten)
        # sigma = torch.exp(self.fc_sigma(x_flatten))
        # z = mu + sigma*self.N.sample(mu.shape)
        # self.kl = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()
        return x_flatten



class CNNDecoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.up3 = (Up(16, 8))
        self.up4 = (Up(8, 4))
        self.up5 = nn.Conv2d(4, 1, 3, padding=1)

    def forward(self, x):
        x3 = x.view((NETS_BATCHSIZE, 16, 16, 16))
        x = self.up3(x3)
        x = self.up4(x)
        x = self.up5(x)
        return x

In [ ]:
class CNNVAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn_encoder = CNNEncoder()
        self.cnn_decoder = CNNDecoder()
    def forward(self, x):
        x = self.cnn_encoder(x)
        x = self.cnn_decoder(x)
        return x

In [ ]:
class CNNVAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.down1 = nn.Linear(64 * 64, 32 * 32)
        self.up3 = nn.Linear(32 * 32, 64 * 64)

        # self.fc_sigma = nn.Linear(32 * 32, 32 * 32)
        # self.fc_mu = nn.Linear(32 * 32, 32 * 32)

        # self.N = torch.distributions.Normal(0, 1)
        # self.N.loc = self.N.loc.to(device)
        # self.N.scale = self.N.scale.to(device)
        self.kl = 0

    def forward(self, x):
        x = x.view(NETS_BATCHSIZE, 64 * 64)
        x = F.relu(self.down1(x))
        # x = F.relu(self.down3(x))

        # mu = self.fc_mu(x)
        # sigma = torch.exp(self.fc_sigma(x))
        # x = mu + sigma*self.N.sample(mu.shape)
        # self.kl = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()

        x = self.up3(x)
        x = x.view(NETS_BATCHSIZE, 1, 64, 64)
        return x

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.down1 = nn.Linear(64 * 64, 45 * 45)

    def forward(self, x):
        x = x.view(NETS_BATCHSIZE, 64 * 64)
        x = F.relu(self.down1(x))
        return x

class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.up1 = nn.Linear(45 * 45, 64 * 64)

    def forward(self, x):
        x = self.up1(x)
        x = x.view(NETS_BATCHSIZE, 1, 64, 64)
        return x

class AE(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc_sigma = nn.Linear(45 * 45, 45 * 45)
        self.fc_mu = nn.Linear(45 * 45, 45 * 45)

        self.N = torch.distributions.Normal(0, 1)
        self.N.loc = self.N.loc.to(device)
        self.N.scale = self.N.scale.to(device)
        self.kl = 0

    def forward(self, x):
        mu = self.fc_mu(x)
        sigma = torch.exp(self.fc_sigma(x))
        x = mu + sigma*self.N.sample(mu.shape)
        self.kl = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()
        return x

In [ ]:
class AE_VAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.ae = torch.load("fitted_model.pt")
        self.ae.eval()
        self.vae = VAE()
        self.vae.train()

    def forward(self, x):
        with torch.no_grad():
            x = self.ae.encoder(x)
        x = self.vae(x)
        with torch.no_grad():
            x = self.ae.decoder(x)
        return x

In [ ]:
class AE_VAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.ae = AE()
        self.vae = VAE()

    def forward(self, x):
        x = self.ae.encoder(x)
        x = self.vae(x)
        x = self.ae.decoder(x)
        return x

In [ ]:
# class CNNVAE(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.down1 = nn.Linear(64 * 64, 45 * 45)
#         self.up3 = nn.Linear(45 * 45, 64 * 64)

#     def forward(self, x):
#         x = x.view(NETS_BATCHSIZE, 1, 64 * 64)
#         x = F.relu(self.down1(x))
#         x = self.up3(x)
#         x = x.view(NETS_BATCHSIZE, 1, 64, 64)
#         return x

In [ ]:
def valid_model(model, valid_loader):
    mse = 0
    smape = 0
    model.eval()
    for data in valid_loader:
        x, y = data
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            output = model(x)
        mse += ((output.T - y) ** 2).sum()
        smape += (output - y).abs() / (output.abs() + y.abs()) / 2
    return mse / len(valid_loader), smape / len(valid_loader)

In [ ]:
def valid_cos_sim(model1, model2, valid_loader):
    cos_sim = 0
    with torch.no_grad():
        for data in valid_loader:
            x, y = data
            x, y = x.to(device), y.to(device)
            model1(x)
            model2(x)

            cos_sim += (F.cosine_similarity(model1.emb, model2.emb).sum() / x.shape[0])
            model1.emb.detach()
            model2.emb.detach()
    return cos_sim / len(valid_loader)

In [ ]:
lr = 3e-4
kl_c = 0.000005

In [ ]:
run = wandb.init(
    project="netformer",
    name=f"linear, ae vae, old_dataset, emb=45*45, lr={lr}, kl_c={kl_c}",
    config={
        "learning_rate": lr,
        "d_model": 0,
        "num_heads": 0,
        "num_layers": 0,
        "d_ff": 0,
        "dropout": 0,
        "BATCHSIZE": NETS_BATCHSIZE,
        "data": "random",
        "edge_c": 0
    },
)

cos_sim,▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇███████▇▄▃▁
gen_model_metric,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
loss_mse,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
mape,▅▅▄▃▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▂▅▆█
mse,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
smape,█▆▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▂▁▁▁▂▅▅▆
train_kl,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
train_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
true_model_metric,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
cos_sim,0.32424


In [ ]:
from torch.optim.lr_scheduler import StepLR
cnnvae = AE_VAE()
# cnnvae.to(device)
criterion = nn.MSELoss()
val_criterion = nn.MSELoss()
optimizer = optim.AdamW(cnnvae.parameters(), lr=lr)
# scheduler = StepLR(optimizer, step_size=20, gamma=0.1)

In [ ]:
for epoch in range(100):
    train_metrics = {'train_loss': 0, 'train_kl': 0, 'loss_mse': 0}
    val_metrics = {'val_loss': 0, 'mse':0, 'mape':0, 'true_model_metric': 0, 'gen_model_metric': 0, 'cos_sim': 0, 'smape': 0}

    cnnvae.train()
    for batch in tqdm(train_dataloader):
        data = batch[0].to(device)
        optimizer.zero_grad()
        output = cnnvae(data.view(NETS_BATCHSIZE, 1, 64 * 64)).view(NETS_BATCHSIZE, 1, 64, 64)
        data.detach()
        loss_mse = criterion(output, data)
        loss = loss_mse + cnnvae.vae.kl * kl_c
        loss.backward()
        optimizer.step()
        train_metrics['train_kl'] += cnnvae.vae.kl * kl_c
        train_metrics['train_loss'] += loss.item()
        train_metrics['loss_mse'] += loss_mse.item()

    train_metrics['train_loss'] /= len(train_dataloader)
    train_metrics['loss_mse'] /= len(train_dataloader)
    train_metrics['train_kl'] /= len(train_dataloader)

    print("train: epoch:", epoch,
          "loss:", round(train_metrics['train_loss'], 4),
          "kl:", train_metrics['train_kl'])


    cnnvae.eval()
    for batch in tqdm(valid_dataloader):
        data, net_num, reg = batch
        with torch.no_grad():
            output = cnnvae(data.to(device))
            val_loss = val_criterion(output, data.to(device))

        val_metrics['val_loss'] += val_loss.item()
        for j in range(NETS_BATCHSIZE):
            true_net = torch.load(f"/content/fitted 2/reg{int(reg[j])}/model{int(net_num[j])}.pt", map_location=torch.device('cpu'))
            # true_net = torch.load(f"/content/model{net_num[j]}.pt", map_location=torch.device('cpu'))
            true_net = RNet([layer for layer in true_net.layers])
            net = reverse_converter(output[j][0], true_net)

            net.to(device).eval()
            true_net.to(device).eval()

            true_model_mse, true_model_smape  = (valid_model(true_net, reg_dataloaders[int(reg[j])][1]))
            gen_model_mse, gen_model_smape  = (valid_model(net, reg_dataloaders[int(reg[j])][1]))

            cos_sim = valid_cos_sim(net, true_net, reg_dataloaders[int(reg[j])][1])

            val_metrics['smape'] += gen_model_smape
            val_metrics['mse'] += (true_model_mse - gen_model_mse)**2
            val_metrics['mape'] += (true_model_mse - gen_model_mse).abs()/gen_model_mse
            val_metrics['true_model_metric'] += true_model_mse
            val_metrics['gen_model_metric'] += gen_model_mse
            val_metrics['cos_sim'] += cos_sim

    val_metrics['smape'] /= (len(valid_dataloader) * NETS_BATCHSIZE)
    val_metrics['cos_sim'] /= (len(valid_dataloader) * NETS_BATCHSIZE)
    val_metrics['val_loss'] /= (len(valid_dataloader))
    val_metrics['mse'] /= (len(valid_dataloader) * NETS_BATCHSIZE)
    val_metrics['mape'] /= (len(valid_dataloader) * NETS_BATCHSIZE)
    val_metrics['true_model_metric'] /= (len(valid_dataloader) * NETS_BATCHSIZE)
    val_metrics['gen_model_metric'] /= (len(valid_dataloader) * NETS_BATCHSIZE)

    print(val_metrics['true_model_metric'], val_metrics['gen_model_metric'])
    print("loss:", round(val_metrics['val_loss'], 4))

    wandb.log(val_metrics | train_metrics)

  0%|          | 0/40 [00:00<?, ?it/s]<ipython-input-14-c7d25f251431>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  nets[i] = torch.tensor(elem[0])
100%|██████████| 40/40 [00:12<00:00,  3.23it/s]


train: epoch: 0 loss: 1.8613 kl: tensor(38855.1797, grad_fn=<DivBackward0>)


  0%|          | 0/12 [00:00<?, ?it/s]<ipython-input-100-c013130d0f30>:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  true_net = torch.load(f"/content/fitted 2/reg{int(re

tensor(4968.9473) tensor(35056.2891)
loss: 1.6944


100%|██████████| 40/40 [00:12<00:00,  3.24it/s]


...

tensor(4968.9473) tensor(nan)
loss: nan


100%|██████████| 40/40 [00:16<00:00,  2.36it/s]


train: epoch: 99 loss: nan kl: tensor(nan, grad_fn=<DivBackward0>)


100%|██████████| 12/12 [00:38<00:00,  3.18s/it]

tensor(4968.9473) tensor(nan)
loss: nan


In [ ]:
'sdkjfdksj'

'sdkjfdksj'

In [ ]:
torch.save(cnnvae, 'fitted_model_07.pt')

In [ ]:
cnnvae = torch.load('fitted_model_07.pt')

<ipython-input-76-de66ffa3bb94>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cnnvae = torch.load('fitted_model_07.pt')


In [ ]:
cnnvae

AE_VAE(
  (ae): AE(
    (encoder): Encoder(
      (down1): Linear(in_features=4096, out_features=2025, bias=True)
    )
    (decoder): Decoder(
      (up1): Linear(in_features=2025, out_features=4096, bias=True)
    )
  )
  (vae): VAE(
    (fc_sigma): Linear(in_features=2025, out_features=2025, bias=True)
    (fc_mu): Linear(in_features=2025, out_features=2025, bias=True)
  )
)

In [ ]:
N = torch.distributions.Normal(0, 1)

In [ ]:
N.sample((1, 2025))

tensor([[ 0.2061, -2.1742,  0.9353,  ...,  0.3444,  1.1068, -2.7100]])

In [ ]:
NETS_BATCHSIZE = 1
cnnvae.ae.decoder(N.sample((1, 2025)))[0][0][:10, :10]

tensor([[ 5.8646e-01,  2.0577e-01,  8.0944e-01,  8.7942e-01, -1.3159e-01,
         -4.7344e-02,  6.7567e-01, -3.3706e-01,  2.3950e-02,  1.3367e-01],
        [-6.3919e-01, -4.8537e-01, -7.6817e-01, -1.0644e-01, -3.6743e-01,
         -7.9478e-02, -1.1275e+00, -6.5370e-01, -5.3133e-01, -4.3214e-01],
        [ 1.2598e+00,  5.4323e-01,  2.1792e+00,  1.0824e+00,  9.9124e-01,
          2.1665e-01,  1.9723e+00,  1.4416e+00,  1.0375e+00,  1.1934e+00],
        [ 6.5013e-03,  4.6656e-01,  6.8867e-01,  2.9081e-01,  8.6360e-01,
          4.9176e-02,  3.9378e-01,  6.1263e-01,  9.1799e-01,  5.3902e-01],
        [-3.4143e-01, -1.8439e-01, -2.7598e-01, -4.8941e-01,  3.4199e-02,
         -2.2404e-01, -1.3024e-01,  6.8110e-02,  4.5147e-01, -6.6847e-01],
        [ 5.4834e-01,  6.6446e-01,  6.5057e-01,  1.9468e-01,  7.3856e-01,
          9.5361e-02,  6.2071e-01,  4.9931e-01,  1.7214e-01,  6.2273e-01],
        [-4.3661e-01, -4.3044e-01, -7.9496e-01, -4.1557e-01, -6.2904e-01,
         -8.1536e-02, -2.0586e-0

In [ ]:
act_map = {nn.ReLU: 2,
           nn.Tanh: 3,
           nn.Sigmoid: 1}

def matrix_converter(net):

    matrix = torch.zeros(64, 64) + torch.full((64, 64), 0)
    count_matrix = torch.zeros(64, 64)
    dims = [0, 0]
    last_layer = 0
    layer_count = 1

    if net.__class__ != nn.Sequential:
        net = net.layers
    for i in range(len(net)):
        layer = net[i]
        if layer.__class__ in act_map.keys():
            for _ in range(last_layer):
                matrix[dims[0], dims[1]] = act_map[layer.__class__]
                count_matrix[dims[0], dims[1]] = layer_count
                dims[0] += 1
                dims[1] += 1
            layer_count += 1
        else:
            matrix[dims[0]:dims[0]+layer.out_features, dims[1]:dims[1]+layer.in_features] = layer.weight
            count_matrix[dims[0]:dims[0]+layer.out_features, dims[1]:dims[1]+layer.in_features] = layer_count * torch.eye(layer.out_features, layer.in_features)
            dims[0] += layer.out_features
            dims[1] += layer.in_features
            last_layer = layer.out_features
            layer_count += 1
    return matrix, count_matrix

In [ ]:
net = Net(10, 1)
net

Net(
  (layers): Sequential(
    (0): Linear(in_features=10, out_features=6, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=6, out_features=8, bias=False)
    (3): Tanh()
    (4): Linear(in_features=8, out_features=4, bias=False)
    (5): Tanh()
    (6): Linear(in_features=4, out_features=1, bias=True)
  )
)

In [ ]:
matrix_converter(net)[1][:20, :18]

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0.],
        [0., 0.,

In [ ]:
matrix_converter(net)[0]

tensor([[ 0.0780, -0.0778, -0.1396,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0427, -0.1252, -0.0733,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2523,  0.1257,  0.0805,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<CopySlices>)

In [ ]:
torch.eye(10, 8)

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])